In [29]:
import numpy as np

def I(valor, total):
    valor = int(valor)
    total = int(total)
    if valor <= 0 or total <= 0 or valor > total:
        return 0
    else:
        return -valor / total * np.log2(valor / total)


In [30]:
import pandas as pd
import math
from collections import Counter
import treelib

def ID3(df, coluna_classe):
    ganho = [0] * (len(df.columns) - 1)

    total = len(df)

    #print(df.columns)
    enClasse = 1


    #entropia classe
    enClasse = 0
    for classe, freq in df[coluna_classe].value_counts().items():
        enClasse += I(freq, total)

    maiorGanho = [0,0]
    #entropia atributo
    for coluna in df.columns:
        #olha quantas alternativas no atributo
        if(coluna != coluna_classe):
            entropia = 0
            contAtr = df[[coluna]].value_counts()
            for atributo, qtd in contAtr.items():
                abc = 0
                lista = df[[coluna_classe]].value_counts()
                for valor, quantidade in lista.items():
                    
                    subconjunto = df[(df[coluna] == atributo[0]) & (df[coluna_classe] == valor[0])]
                    total_sub = len(subconjunto)
                    abc += I(total_sub,qtd)
                gv = len(df)
                entropia += (qtd/gv)*abc
            ganho = enClasse - entropia
            if(ganho>maiorGanho[0]):
                #print("testestestestesttestestestst")
                maiorGanho[0] = ganho
                maiorGanho[1] = coluna

    #print(maiorGanho[1])
    return maiorGanho

In [31]:
import pandas as pd
import math
from collections import Counter
import treelib

def geraArvore(df, coluna_classe, tree, parent=None, ramo_valor=None):
    maiorGanho = ID3(df, coluna_classe)

    # Verifica se ainda há ganho de informação
    if maiorGanho[0] != 0:
        atributo = str(maiorGanho[1])  # Certifique-se de que é string
        # Cria nó do atributo
        if parent is None:
            node_id = str(atributo)
            tree.create_node(tag=str(atributo), identifier=node_id)
        else:
            node_id = str(f"{atributo}_{len(tree.nodes)}")
            tree.create_node(tag=f"{str(ramo_valor)} → {str(atributo)}", identifier=node_id, parent=str(parent))

        # Para cada valor do atributo, cria ramos
        for valor in df[atributo].unique():
            df_filtrado = df[df[atributo] == valor]

            # Verifica se todos têm a mesma classe (nó folha)
            classes = df_filtrado[coluna_classe].unique()
            if len(classes) == 1:
                classe_final = str(classes[0])
                folha_id = str(f"{classe_final}_{len(tree.nodes)}")
                tree.create_node(tag=f"{str(valor)} → {classe_final}", identifier=folha_id, parent=node_id)
            else:
                df_sub = df_filtrado.drop(columns=[atributo])
                geraArvore(df_sub, coluna_classe, tree, parent=node_id, ramo_valor=str(valor))
    else:
        # Caso sem ganho de informação, criar nó com classe majoritária
        classe_majoritaria = str(df[coluna_classe].mode()[0])
        folha_id = str(f"{classe_majoritaria}_{len(tree.nodes)}")
        tag = f"{str(ramo_valor)} → {classe_majoritaria}" if ramo_valor else classe_majoritaria
        tree.create_node(tag=tag, identifier=folha_id, parent=str(parent))


In [32]:
import pandas as pd
import math
from collections import Counter
from treelib import Tree


# Abrindo o arquivo CSV
df = pd.read_csv('train.csv',sep = ",")

# Remover colunas que não são úteis para classificação
df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

# Preencher valores ausentes:
# 'Age' será preenchido com a mediana
df['Age'].fillna(df['Age'].median(), inplace=True)

# 'Embarked' será preenchido com o valor mais frequente
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Convertendo variáveis categóricas em numéricas ou discretas:

# 'Sex' (categórica para nominal)
df['Sex'] = df['Sex'].map({'male': 'Homem', 'female': 'Mulher'})

# Discretizar a idade em faixas
df['Age'] = pd.cut(df['Age'], bins=[0, 12, 18, 30, 50, 80], labels=['Criança', 'Adolescente', 'Jovem', 'Adulto', 'Idoso'])

# Discretizar a tarifa (Fare)
df['Fare'] = pd.qcut(df['Fare'], q=4, labels=['Baixo', 'Médio', 'Alto', 'Muito alto'])

# Mapear 'Embarked' para nomes
df['Embarked'] = df['Embarked'].map({'S': 'Southampton', 'C': 'Cherbourg', 'Q': 'Queenstown'})

coluna_classe = "Survived"
# TEste de abertura
#print(df.head()) 
  
#colocar na árvore
tree = Tree()
geraArvore(df,coluna_classe, tree)
tree.show()

C:\Users\Eu\AppData\Local\Temp\ipykernel_15992\1843324328.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
C:\Users\Eu\AppData\Local\Temp\ipykernel_15992\1843324328.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

Sex
├── Homem → Age
│   ├── Adolescente → Parch
│   │   ├── 0 → Fare
│   │   │   ├── Alto → 0
│   │   │   ├── Baixo → 0
│   │   │   ├── Muito alto → 0
│   │   │   └── Médio → Pclass
│   │   │       ├── 2 → 0
│   │   │       └── 3 → 0
│   │   ├── 1 → 0
│   │   ├── 2 → Pclass
│   │   │   ├── 1 → 1
│   │   │   └── 3 → 0
│   │   └── 3 → 0
│   ├── Adulto → Pclass
│   │   ├── 1 → Fare
│   │   │   ├── Alto → Parch
│   │   │   │   ├── 0 → Embarked
│   │   │   │   │   ├── Cherbourg → 1
│   │   │   │   │   └── Southampton → 1
│   │   │   │   └── 1 → 0
│   │   │   ├── Baixo → 0
│   │   │   └── Muito alto → SibSp
│   │   │       ├── 0 → Embarked
│   │   │       │   ├── Cherbourg → Parch
│   │   │       │   │   ├── 0 → 0
│   │   │       │   │   └── 1 → 1
│   │   │       │   └── Southampton → 0
│   │   │       ├── 1 → Parch
│   │   │       │   ├── 0 → Embarked
│   │   │       │   │   ├── Cherbourg → 1
│   │   │       │   │   └── Southampton → 0
│   │   │       │   ├── 1 → Embarked
│   │   │       │ 